In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/shtormzxc/50k_25cats_FINAL.pkl


In [2]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pickle
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score, classification_report
from sklearn.preprocessing import LabelEncoder
from transformers import BertModel, AutoTokenizer
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [3]:
DATA = pd.read_pickle('/kaggle/input/shtormzxc/50k_25cats_FINAL.pkl')
DATA

,text,topic
347,источник рецепты леди mail ru ингредиенты пече...,рецепты
338,выбор косметики и инвентаря для качественной о...,мода
324057,европейские странычлены оэср с января по авгус...,экономика
44746,российский футболист иван заборовский выступаю...,спорт
392947,в суд направлено уголовное дело в отношении од...,общество
...,...,...
330332,тем кто живет в ожидании повестки не стоит ухо...,забота о себе
463,в этом сезоне серый новый черный он встречалс...,мода
321949,лидер партии яблоко сергей митрохин направил в...,общество
1544,источник рецепты леди mail ru ингредиенты карт...,рецепты


In [4]:
len(DATA['topic'].value_counts().to_dict().keys())

25

In [5]:
lbl_enc = LabelEncoder()
DATA['topic'] = lbl_enc.fit_transform(DATA['topic'])

In [6]:
DATA['topic'].value_counts()

topic
24    2301
19    2301
16    2300
10    2300
23    2300
4     2300
0     2300
5     2300
17    2300
2     2300
15    2300
21    2300
9     2300
7     2296
8     2286
22    2192
13    2181
6     2152
20    2095
18    2008
3     1881
14    1253
11     839
12     242
1       86
Name: count, dtype: int64

In [7]:
device = 'cuda'

class ClassifierNN1(nn.Module):
    def __init__(self, N_classes=25):
        super(ClassifierNN1, self).__init__()
        self.bert = BertModel.from_pretrained('gblssroman/mailru-bert-base').to(device)
        self.tokenizer_bert = AutoTokenizer.from_pretrained('gblssroman/mailru-bert-base')
        self.rubert = BertModel.from_pretrained('DeepPavlov/rubert-base-cased').to(device)
        self.tokenizer_rubert = AutoTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')
        
        self.fc_bert = nn.Sequential(
            nn.Linear(768, 256),
            nn.BatchNorm1d(num_features=256),
            nn.GELU()
        )
        self.fc_rubert = nn.Sequential(
            nn.Linear(768, 256),
            nn.BatchNorm1d(num_features=256),
            nn.GELU()
        )
        self.fc = nn.Sequential(
            nn.Linear(512, N_classes)
        )
        
    def mean_pooling(self, model_output, attention_mask):
        token_embeddings = model_output[0]  # each token embeds
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
        sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
        return sum_embeddings / sum_mask
    
    def freeze_berts(self):
        for i in [self.bert, self.rubert]:
            for param in i.parameters():
                param.requires_grad = False

    def forward(self, x):
        y = self.tokenizer_bert(x, padding=True, truncation=True, max_length=512, return_tensors="pt").to(device)
        z = self.tokenizer_rubert(x, padding=True, truncation=True, max_length=512, return_tensors="pt").to(device)
        y_emb, z_emb = self.bert(**y), self.rubert(**z)
        y_emb_pooled = self.mean_pooling(y_emb, y['attention_mask']).to(device)
        z_emb_pooled = self.mean_pooling(z_emb, z['attention_mask']).to(device)
        y_fc = self.fc_bert(y_emb_pooled)
        z_fc = self.fc_rubert(z_emb_pooled)
        yz = torch.cat((y_fc, z_fc), dim=1)
        out = self.fc(yz)
        return out
    
model1 = ClassifierNN1()
model1.freeze_berts()
model1.to(device)

config.json:   0%|          | 0.00/647 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of BertModel were not initialized from the model checkpoint at gblssroman/mailru-bert-base and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/184k [00:00<?, ?B/s]

...

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequence

tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

ClassifierNN1(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_aff

In [8]:
class TextDataset(Dataset):
    def __init__(self, text, labels):
        self.text = text.to_numpy()
        self.labels = torch.tensor(labels.to_numpy(), dtype=torch.float32)
    def __getitem__(self, idx):
        out = {}
        out['text'] = self.text[idx]
        out['labels'] = self.labels[idx]
        return out
    def __len__(self):
        return len(self.labels)

In [9]:
X = DATA['text']
Y = DATA['topic']
Xtrain, Xval, Ytrain, Yval = train_test_split(X, Y, stratify=Y, test_size=0.25, random_state=666, shuffle=True)

In [10]:
BATCH_SIZE = 128

trainset = TextDataset(Xtrain, Ytrain)
valset = TextDataset(Xval, Yval)
trainloader = DataLoader(trainset, batch_size=BATCH_SIZE)
valloader = DataLoader(valset, batch_size=BATCH_SIZE, shuffle=False)

In [11]:
EPOCHS = 3
LR = 6e-4

optimizer = torch.optim.AdamW(params=model1.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=0.7)
loss = nn.CrossEntropyLoss()
sfmax = nn.Softmax(dim=1)

f1train, f1val = [], []
losstrain, lossval = [], []

for epoch in range(EPOCHS):
    model1.train()
    model1.freeze_berts()
    
    with tqdm(trainloader, unit='batch') as tepoch:
        for batch in tepoch:
            text = batch['text']
            labels = batch['labels'].type(torch.LongTensor).to(device)
            
            optimizer.zero_grad()
            logits = model1(text).to(device)
            curr_loss = loss(logits, labels)
            preds = torch.argmax(sfmax(logits), dim=1, keepdim=True).squeeze()
            
            f1 = f1_score(preds.cpu().detach().numpy(), labels.cpu().detach().numpy(),
                          average='weighted')
            lossitem = curr_loss.item()
            
            f1train.append(f1)
            losstrain.append(lossitem)
            
            curr_loss.backward()
            optimizer.step()
            
            tepoch.set_postfix({'loss': lossitem, 'f1_weighted': f1})
            
    scheduler.step()
    model1.eval()
    model1.freeze_berts()
    with tqdm(valloader, unit='batch') as tepoch:
        for batch in tepoch:
            text = batch['text']
            labels = batch['labels'].type(torch.LongTensor).to(device)
            
            logits = model1(text).to(device)
            curr_loss = loss(logits, labels)
            preds = torch.argmax(sfmax(logits), dim=1, keepdim=True).squeeze()
            
            f1 = f1_score(preds.cpu().detach().numpy(), labels.cpu().detach().numpy(),
                          average='weighted')
            lossitem = curr_loss.item()
            
            f1val.append(f1)
            lossval.append(lossitem)
            
            tepoch.set_postfix({'loss': lossitem, 'f1_weighted': f1})
    

100%|██████████| 97/97 [08:01<00:00,  4.97s/batch, loss=0.435, f1_weighted=0.869]


In [ ]:
model1

In [13]:
file_path = 'mailru_bert_mailru_bert_base.pth'

torch.save(model1.state_dict(), file_path)

In [ ]:
with torch.no_grad():
    for i in valloader:
        print(i)
        break

In [36]:
all_preds = []
all_labels = []
with torch.no_grad():
    with tqdm(valloader, unit='batch') as tepoch:
        for batch in tepoch:
            text = batch['text']
            labels = batch['labels'].type(torch.LongTensor).to(device)

            logits = model1(text).to(device)
            curr_loss = loss(logits, labels)
            preds = sfmax(logits)

            all_preds.extend(preds.cpu().detach().numpy())
            all_labels.extend(labels.cpu().detach().numpy())

100%|██████████| 97/97 [08:01<00:00,  4.97s/batch]


In [39]:
np.save('preds3', np.array(all_preds))

In [40]:
np.save('labels', np.array(all_labels))

In [41]:
np.array(all_preds).shape

(12354, 25)